In [1]:
import torch
from torch.nn import functional as F
import numpy as np
import meshio

from torchmcubes import marching_cubes, grid_interp

import distillation

In [2]:
nsr = torch.load("nsr000.pt")

C:\Users\andrewhc\AppData\Local\Temp\ipykernel_20892\633730493.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  nsr = torch.load("nsr000.pt")


In [3]:
fg_sdf = nsr["s.fg_sdf"][0][0]

In [5]:
# verts, faces = marching_cubes(fg_sdf, thresh=0.0)

# # removes duplicate vertices
# verts, reverse_ind = torch.unique(verts, dim=0, return_inverse=True)
# faces = reverse_ind[faces]

verts, faces = distillation.compute_mesh(fg_sdf)

In [6]:
mesh = meshio.Mesh(
    verts.detach().cpu().numpy().tolist(),
    [("triangle", faces.detach().cpu().numpy().tolist())]  # + albedo (color) & roughness
)
mesh.write("calculator.obj")

In [7]:
normal = torch.zeros(verts.shape)
albedo = torch.ones(verts.shape[0])  # parameter
roughness = torch.ones(verts.shape[0])  # parameter

In [8]:
# # Fx(A,B,C) -> Fx(A,B,C)x(X,Y,Z)
# vf = verts[faces]
# face_norm = torch.linalg.cross(vf[:,1]-vf[:,0], vf[:,2]-vf[:,0])

# # index_put_: supports simulataneous summation for duplicate indices in faces
# vert_norm = torch.zeros(verts.shape, device="cuda")
# vert_norm.index_put_(indices=(faces.flatten(),),
#                      values=torch.repeat_interleave(face_norm, 3, dim=0),
#                      accumulate=True)
# vert_norm = F.normalize(vert_norm)
distillation.compute_normal(verts, faces)

tensor([[ 1.6484, -3.1133],
        [ 1.5569,  2.9234],
        [ 1.1659,  3.0829],
        ...,
        [ 1.0511,  0.1610],
        [ 1.3887,  0.5361],
        [ 1.5682, -0.1042]], device='cuda:0')

In [36]:
vert_norm

tensor([[-0.9966, -0.0775, -0.0282],
        [-0.9762,  0.0139,  0.2164],
        [-0.9176,  0.3939,  0.0539],
        ...,
        [ 0.8568,  0.4966,  0.1391],
        [ 0.8455,  0.1811,  0.5023],
        [ 0.9946,  0.0025, -0.1040]], device='cuda:0')

In [40]:
torch.stack([
    # THETA
    torch.acos(vert_norm[:,1] / torch.linalg.norm(vert_norm, dim=1)),  #*(180.0/np.pi)
    # PHI
    torch.sign(vert_norm[:,2])*torch.acos(vert_norm[:,0] / torch.linalg.norm(vert_norm[:,[0,2]], dim=1)),   #*(180.0/np.pi)
], dim=1)

tensor([[ 1.6484, -3.1133],
        [ 1.5569,  2.9234],
        [ 1.1659,  3.0829],
        ...,
        [ 1.0511,  0.1610],
        [ 1.3887,  0.5361],
        [ 1.5682, -0.1042]], device='cuda:0')

In [9]:
verts.shape, vert_norm.shape, albedo.shape, roughness.shape

(torch.Size([3643, 3]),
 torch.Size([3643, 3]),
 torch.Size([3643]),
 torch.Size([3643]))

In [12]:
sg_params = torch.ones((256, 6), device="cuda")  # 256x[R,G,B,λ,θp,φp]

In [32]:
θp, φp = sg_params[:, 4], sg_params[:, 5]
spherical_dot = lambda θv, φv: torch.sin(θv)*torch.sin(θp)*torch.cos(φv-φp)+torch.cos(θv)*torch.cos(θp)
# (sg_params[:, 4:6]*torch.tensor([-1,5], device="cuda")).sum(axis=1)

lights = lambda θv, φv: sg_params[:, 0:3]*torch.exp(sg_params[:, 3]*(spherical_dot(θv, φv)-1))[:, torch.newaxis]

result = lights(
    torch.tensor(-1, device="cuda"),
    torch.tensor(5, device="cuda"),
).sum(axis=0)



In [41]:
result

tensor([200.3231, 200.3231, 200.3231], device='cuda:0')

In [19]:
θp, φp = sg_params[:, 4], sg_params[:, 5]
spherical_dot = lambda θv, φv: torch.sin(θv)*torch.sin(θp)*torch.cos(φv-φp)+torch.cos(θv)*torch.cos(θp)
# (sg_params[:, 4:6]*torch.tensor([-1,5], device="cuda")).sum(axis=1)
spherical_dot(torch.tensor(-1, device="cuda"), torch.tensor(5, device="cuda"))

tensor([0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548,
        0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548,
        0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548,
        0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548,
        0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548,
        0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548,
        0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548,
        0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548,
        0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548,
        0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548,
        0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548,
        0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548, 0.7548,
        0.7548, 0.7548, 0.7548, 0.7548, 

In [16]:
θp, torch.sin(θp)*torch.sin(θp)

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,